# This is a basic RNN model Designed to develop evaluation metrics

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
from tqdm import tqdm
import iri_dataset_generator as iri
from training_loop import train_model

SEQUENCE_LENGTH = 10
train, test = iri.load_iri_datasets(path="../training_data/IRI-only.parquet",seq_length=SEQUENCE_LENGTH)
train

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
class FNN(nn.Module):
    def __init__(self):
        super(FNN, self).__init__()
        self.rnn = nn.RNN(input_size=SEQUENCE_LENGTH, hidden_size=30, num_layers=5, batch_first=True)
        self.final = nn.Linear(30, 2)
        
    def forward(self, x):
        h0 = torch.zeros(5, x.size(0), 30).to(x.device)
        out, _ = self.rnn(x, h0)

        # Decode the hidden state of the last time step
        out = self.final(out[:, -1, :])
        return out

In [ ]:
model = FNN()
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

train_model(model, train, test, loss, optimizer, epochs=200, test_every_n=10, batch_size=512)

In [ ]:
from torcheval.metrics import R2Score
from torch.utils.data import DataLoader

train_r2 = R2Score()
test_r2 = R2Score()

device = device = torch.device("cuda")
model.eval()
with torch.no_grad():
    right = 0
    total = 0
    train_data = DataLoader(train, batch_size=256, shuffle=True)
    for _, data in enumerate(train_data):
        inputs, goal = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        train_r2.update(outputs, goal)
    for _, data in enumerate(test_data):
        inputs, goal = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        test_r2.update(outputs, goal)

print(train_r2.compute())
print(test_r2.compute())



In [ ]:
right = 0
wrong = 0


def compute_correctness(predicted, target):
    global right, wrong

    def get_slot(mean):
        boundries = [1.5, 2.68]
        if mean < (boundries[0] - iri.mean_iri) / iri.iri_range:
            return 0
        elif mean < (boundries[1] - iri.mean_iri) / iri.iri_range:
            return 1
        else:
            return 2
        
    if get_slot(predicted.mean()) == get_slot(target.mean()):
        right += 1
    else:
        wrong += 1

device = device = torch.device("cuda")
model.eval()
with torch.no_grad():
    right = 0
    total = 0
    train_data = DataLoader(train, batch_size=256, shuffle=True)
    for _, data in enumerate(train_data):
        inputs, goal = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        for pred, target in zip(outputs, goal):
            compute_correctness(pred, target)
            total += 1
    print(f'Accuracy of the network on the train data: {100 * right / total} %')
    test_data = DataLoader(test, batch_size=256, shuffle=True)
    right = 0
    total = 0
    for _, data in enumerate(test_data):
        inputs, goal = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        for pred, target in zip(outputs, goal):
            compute_correctness(pred, target)
            total += 1
    print(f'Accuracy of the network on the test data: {100 * right / total} %')